In [3]:
import pandas as pd
import os
import numpy as np
base_dir = os.path.dirname(os.path.abspath(''))


# Preprocess legacy files

In [4]:
#load historic matches file in old format
historic_matches = pd.read_csv(os.path.join(base_dir,'data','historic_matches.csv'))


In [14]:
#convert the historic matches to a file where there are columns: mentee_name, mentor_name, date, within_region, match_type
matches = pd.DataFrame()
#convert from wide to condensed
#first get the date list (the unique values before the underscore, not including the mentee name column)
date_list = np.unique([col.split('_')[0] for col in historic_matches.columns])
date_list = date_list[date_list != 'Mentee']
for i,col in enumerate(date_list): #don't include the first column because that is the mentee name
    date = col
    df = historic_matches[['Mentee_Name',date + '_mentorName',date + '_withinRegion']].copy()
    df['round'] = i+1
    df['date'] = date
    #rename columns
    df.rename(columns={'Mentee_Name':'mentee_name',date+'_mentorName':'mentor_name',date+'_withinRegion':'within_region'},inplace=True)

    #combinte each sub df into the main matches df
    matches = pd.concat([matches,df])

#drop nas in mentor_name because that means there was no match
matches = matches.dropna(subset='mentor_name')

#fix a data quality error where sometimes the email is listed after the name eg first last(first.last@email.com)
matches.mentee_name = matches.mentee_name.apply(lambda x: x.split('(')[0])
matches.mentor_name = matches.mentor_name.apply(lambda x: x.split('(')[0])
matches['pairs'] = matches.mentor_name + ' - ' + matches.mentee_name


In [15]:
email_list = pd.read_csv(os.path.join(base_dir,'data','email_list.csv')) 
email_list['full_name'] = email_list['First Name'] + ' ' + email_list['Last Name']
email_list['LAST_CHANGED'] = pd.to_datetime(email_list.LAST_CHANGED, format='%m/%d/%Y %H:%M')
email_list = email_list.sort_values(by='LAST_CHANGED',ascending=False)
#drop duplicates of the same full name by sorting by the last date they changed their info and taking the first entry
email_list['LAST_CHANGED']
email_list = email_list.drop_duplicates(subset='full_name', keep='first')

#load email list to get info about what type of match it was 
mentees = email_list[email_list['Do you want to join Coffee Catchup as a Mentee?'] == 'Yes'].copy()
mentee_name_list = mentees.full_name.unique()
mentors = email_list[email_list['Do you want to join Coffee Catchup as a Mentor?'] == 'Yes'].copy()
mentor_name_list = mentors.full_name.unique()
matches['mentee_type'] = matches.apply(lambda x: 'mentee' if x.mentee_name in mentee_name_list else 'mentor' if x.mentee_name in mentor_name_list else 'error',axis=1)
matches['mentor_type'] = matches.apply(lambda x: 'mentee' if x.mentor_name in mentee_name_list else 'mentor' if x.mentor_name in mentor_name_list else 'error',axis=1)
matches['type'] = matches['mentor_type'] + '-' + matches['mentee_type'] 

In [21]:
#save historic matches to csv with new format
matches[['mentee_name','mentor_name','within_region','round','date','pairs','mentee_type','type']].to_csv(os.path.join(base_dir,'data','historic_matches_v2.csv'),index=False)


AttributeError: 'DataFrame' object has no attribute 'region'